# Library

In [1]:
import numpy as np
import torch
import gpytorch
import argparse
import time
import pickle
import scipy.io as sio

from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

import sys
sys.path.append('..')

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


## Check GPU

In [2]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


# Set Arguments

In [3]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=420, help='size of mini batch')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gp_net_torch', help='rnn, gru, or lstm')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
parser.add_argument('--enable_ros', type=bool, default=False, help='put data into ros')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')

sys.argv = ['']
args = parser.parse_args()

if args.enable_ros:
    rospy.init_node('global_localization_tf_broadcaster_cnn')

# Load Dataset

In [4]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet, cnn_auxiliary
from torchlib.cnn_auxiliary import normalize, denormalize, denormalize_navie, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.test_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate', sampling_rate=1)

[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

100%|██████████| 25367/25367 [00:51<00:00, 491.30it/s]


Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [9]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, BaseModule

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        context_feat = self.global_context(input_data)
        #context_feat = input_data
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r

class GP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points, output_dim=3):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([output_dim])
        )
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=output_dim
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        #self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([3]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([3])),
            batch_shape=torch.Size([3]))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        inducing_points = torch.zeros(3, 1000, 128)
        self.backbone = Backbone()
        self.nn = NN()
        self.gp = GP(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)
        
    def forward(self, input_data):
        dense_feat = self.backbone(input_data)
        #dense_feat = input_data
        output, feature_t, feature_r = self.nn(dense_feat)
        rot_pred = torch.split(output, [3, 4], dim=1)[1] # 4-dimention 
        trans_pred = self.gp(feature_t)
        return trans_pred, rot_pred

class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args,
                 is_training=True, mixed_precision=True,
                 regressor_context_rate = [0.0,0.0], train_rot = False):
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        self.train_rot = train_rot
        self.mixed_precision = mixed_precision
        if self.mixed_precision:
            self.scaler = GradScaler()

        self.disable_requires_grad(self.model.backbone)
        
        if is_training:
            self.optimizer = optim.Adam(self._optimize(regressor_context_rate))
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
            num_data = min(len(dataloader)*args.batch_size,len(dataset))
            self.mll = gpytorch.mlls.PredictiveLogLikelihood(self.model.likelihood, \
                                                         self.model.gp, num_data = num_data)
        else:
            self.disable_requires_grad(self.model)
            
    def _optimize(self,regressor_context_rate):
        # GP
        optimizer = [
                {'params': self.model.gp.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay},
                {'params': self.model.likelihood.parameters(), \
                 'lr': args.learning_rate,'weight_decay':args.weight_decay}]
            
        # NN
        if regressor_context_rate[0]!=0:
            optimizer += [{'params': self.model.nn.global_regressor.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[0],'weight_decay':args.weight_decay}]
            print('Regressor learn rate:',regressor_context_rate[0])
        else:
            self.disable_requires_grad(self.model.nn.global_regressor)
                
        if regressor_context_rate[1]!=0:
            optimizer += [{'params': self.model.nn.global_context.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[1],'weight_decay':args.weight_decay}]
            print('Context learn rate:',regressor_context_rate[1])
            self.train_rot = True
        else:
            self.disable_requires_grad(self.model.nn.global_context)
            
        
        if not self.train_rot and regressor_context_rate[1]==0.0:
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc1_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc2_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc3_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.logits_r)
                
        return optimizer
    
    def train(self,x, y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert trainer.model.training == True
        if self.mixed_precision:
            with autocast():
                single_loss = self._loss(x, y)
            self.scaler.scale(single_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            single_loss = self._loss(x, y)
            single_loss.backward()
            self.optimizer.step()
            
        batch_time = time.time() - start
        
        return float(single_loss), batch_time
    
    def _loss(self,x, y):
        # target
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        # predict
        trans_pred, rot_pred = self.model(x)
        
        # trans loss
        trans_loss = -1.*self.mll(trans_pred, trans_target)
        # rot loss
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_target),dim=1)))
        
        total_loss = trans_loss + args.lamda_weights * rot_loss      
        
        return total_loss
    
    def eval_forward(self,x,y,num_sample = 100,output_denormalize = True):
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        
        # Step 2: forward
        assert trainer.model.training == False
        if self.mixed_precision:
            with autocast():
                trans_prediction, rot_prediction = self.model(x)
            self.scaler.scale(trans_prediction)
            self.scaler.scale(rot_prediction)
        else:
            trans_prediction, rot_prediction = self.model(x)
            
        trans_prediction, trans_mean, trans_var = self._eval_gp(trans_prediction)
        
        if args.is_normalization and output_denormalize:
            trans_prediction = denormalize_navie(trans_prediction, self.norm_mean, self.norm_std)
            trans_mean = denormalize_navie(trans_mean, self.norm_mean, self.norm_std)
            trans_var = trans_var.mul(self.norm_std)
        
        samples = self._sample(trans_mean, trans_var, num_sample)
            
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        
        return trans_prediction, rot_prediction, trans_target, rot_target, samples
    
    def _sample(self, mean, var, num_sample = 100):
        dist = torch.distributions.Normal(mean, var)
        samples = dist.sample([num_sample])
        return samples
    
    def _eval_gp(self, trans_pred):
        c_mean, c_var = trans_pred.mean, trans_pred.variance
        y_mean, y_var = self.model.likelihood(trans_pred).mean, self.model.likelihood(trans_pred).variance
        
        return y_mean, c_mean, c_var

In [10]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,mixed_precision=False,is_training = False)
trainer.load_model('model-fast-zero-mean.pth')

Successfully loaded model to TITAN Xp.


# Initialize

In [11]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

In [12]:
trainer.model.eval()

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    trans_pred, rot_pred, trans_gt, rot_gt, samples = trainer.eval_forward(x,y)
    
    # transform data
    trans_pred = trans_pred.cpu().numpy()
    rot_pred = rot_pred.cpu().numpy()
    trans_gt = trans_gt.cpu().numpy()
    rot_gt = rot_gt.cpu().numpy()
    
    end = time.time()
    
    if args.enable_ros:
        particles = PoseArray()
        particles.header.stamp = rospy.Time.now()
        particles.header.frame_id = 'world'
        for s in samples:
            pose = Pose()
            [pose.position.x, pose.position.y, pose.position.z] = s
            [pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w] = rot_pred[0]
            particles.poses.append(pose)
        particles_pub.publish(particles)

        [px_pred, py_pred, pz_pred] = trans_pred[0]
        [qx_pred, qy_pred, qz_pred, qw_pred] = rot_pred[0]

        br.sendTransform((px_pred, py_pred, pz_pred),
                         (qx_pred, qy_pred, qz_pred, qw_pred), rospy.Time.now(),
                         "estimation", "world")

        [px_gt, py_gt, pz_gt] = trans_gt[0]
        [qx_gt, qy_gt, qz_gt, qw_gt] = rot_gt[0]

        br.sendTransform((px_gt, py_gt, pz_gt),
                         (qx_gt, qy_gt, qz_gt, qw_gt),
                         rospy.Time.now(), "gt", "world")

        timestamp = rospy.Time.now()

        nn_pose_msg = Odometry()
        nn_pose_msg.header.frame_id = 'world'
        nn_pose_msg.header.stamp = timestamp
        nn_pose_msg.child_frame_id = 'base_link'
        nn_pose_msg.pose.pose.position.x = px_pred
        nn_pose_msg.pose.pose.position.y = py_pred
        nn_pose_msg.pose.pose.position.z = pz_pred
        [nn_pose_msg.pose.pose.orientation.x, nn_pose_msg.pose.pose.orientation.y, nn_pose_msg.pose.pose.orientation.z, nn_pose_msg.pose.pose.orientation.w] = [qx_pred, qy_pred, qz_pred, qw_pred]

        conv = np.zeros((6,6), dtype=np.float32)
        [conv[0][0], conv[1][1], conv[2][2]] = trans_cov[0]
        nn_pose_msg.pose.covariance = conv.flatten().tolist()
        nn_pose_pub.publish(nn_pose_msg)

        bridge = CvBridge()

        bird_view_img_msg = bridge.cv2_to_imgmsg(np.asarray(x[0].cpu(), dtype=np.float32), encoding="passthrough")
        stamp_now = rospy.Time.now()
        bird_view_img_msg.header.stamp = stamp_now

        bird_view_pub.publish(bird_view_img_msg)

        rospy.sleep(.0)
        cv2.waitKey(0)

        count += 1
    else:
        count += y.shape[0]
    
    trans_preds += [x for x in trans_pred]
    rot_preds += [x for x in rot_pred]
    trans_gts += [x for x in trans_gt]
    rot_gts += [x for x in rot_gt]

    trans_error = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
    rot_error_1 = np.arccos(np.sum(np.multiply(rot_pred,rot_gt),axis=1))/math.pi*180
    rot_error_2 = np.arccos(np.sum(np.multiply(rot_pred,-rot_gt),axis=1))/math.pi*180
    rot_error = np.minimum(rot_error_1,rot_error_2)

    trans_errors += [x for x in trans_error]
    rot_errors += [x for x in rot_error]

    total_trans_error += np.sum(trans_error)
    total_rot_error += np.sum(rot_error)
    
    display = 1

    if b % display == 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             (b+1)*args.batch_size,
            len(dataloader)*args.batch_size,
            total_trans_error / count,
            total_rot_error / count,
            end - start))

420/183960, translation error = 3.152, rotation error = 2.385, time/batch = 4.811
840/183960, translation error = 8.310, rotation error = 3.766, time/batch = 1.054
1260/183960, translation error = 6.764, rotation error = 3.446, time/batch = 1.136
1680/183960, translation error = 5.940, rotation error = 3.279, time/batch = 1.054
2100/183960, translation error = 6.777, rotation error = 3.423, time/batch = 1.052
2520/183960, translation error = 6.547, rotation error = 3.498, time/batch = 1.058
2940/183960, translation error = 6.113, rotation error = 3.642, time/batch = 1.054
3360/183960, translation error = 5.526, rotation error = 3.658, time/batch = 1.051
3780/183960, translation error = 5.147, rotation error = 3.602, time/batch = 1.061
4200/183960, translation error = 5.051, rotation error = 3.730, time/batch = 1.059
4620/183960, translation error = 4.761, rotation error = 3.701, time/batch = 1.061
5040/183960, translation error = 4.557, rotation error = 3.723, time/batch = 1.069
5460/1

41580/183960, translation error = 4.372, rotation error = 4.059, time/batch = 1.072
42000/183960, translation error = 4.340, rotation error = 4.053, time/batch = 1.069
42420/183960, translation error = 4.310, rotation error = 4.057, time/batch = 1.074
42840/183960, translation error = 4.286, rotation error = 4.048, time/batch = 1.072
43260/183960, translation error = 4.356, rotation error = 4.055, time/batch = 1.067
43680/183960, translation error = 5.838, rotation error = 4.222, time/batch = 1.067
44100/183960, translation error = 7.924, rotation error = 4.867, time/batch = 1.079
44520/183960, translation error = 9.909, rotation error = 5.271, time/batch = 1.072
44940/183960, translation error = 9.909, rotation error = 5.252, time/batch = 1.073
45360/183960, translation error = 9.851, rotation error = 5.235, time/batch = 1.075
45780/183960, translation error = 9.776, rotation error = 5.220, time/batch = 1.073
46200/183960, translation error = 9.732, rotation error = 5.226, time/batch 

82740/183960, translation error = 8.988, rotation error = 5.131, time/batch = 1.081
83160/183960, translation error = 8.953, rotation error = 5.127, time/batch = 1.078
83580/183960, translation error = 8.915, rotation error = 5.122, time/batch = 1.070
84000/183960, translation error = 8.880, rotation error = 5.121, time/batch = 1.068
84420/183960, translation error = 8.927, rotation error = 5.127, time/batch = 1.070
84840/183960, translation error = 8.901, rotation error = 5.111, time/batch = 1.073
85260/183960, translation error = 8.874, rotation error = 5.097, time/batch = 1.072
85680/183960, translation error = 8.844, rotation error = 5.082, time/batch = 1.077
86100/183960, translation error = 8.811, rotation error = 5.067, time/batch = 1.072
86520/183960, translation error = 8.776, rotation error = 5.062, time/batch = 1.069
86940/183960, translation error = 8.756, rotation error = 5.049, time/batch = 1.075
87360/183960, translation error = 8.773, rotation error = 5.077, time/batch 

123480/183960, translation error = 9.720, rotation error = 5.354, time/batch = 1.074
123900/183960, translation error = 9.694, rotation error = 5.354, time/batch = 1.078
124320/183960, translation error = 9.675, rotation error = 5.353, time/batch = 1.076
124740/183960, translation error = 9.650, rotation error = 5.348, time/batch = 1.075
125160/183960, translation error = 9.624, rotation error = 5.338, time/batch = 1.074
125580/183960, translation error = 9.598, rotation error = 5.336, time/batch = 1.074
126000/183960, translation error = 9.573, rotation error = 5.334, time/batch = 1.069
126420/183960, translation error = 9.571, rotation error = 5.346, time/batch = 1.064
126840/183960, translation error = 9.559, rotation error = 5.352, time/batch = 1.069
127260/183960, translation error = 9.533, rotation error = 5.344, time/batch = 1.078
127680/183960, translation error = 9.510, rotation error = 5.336, time/batch = 1.074
128100/183960, translation error = 9.512, rotation error = 5.341,

164220/183960, translation error = 11.057, rotation error = 6.285, time/batch = 1.072
164640/183960, translation error = 11.044, rotation error = 6.281, time/batch = 1.075
165060/183960, translation error = 11.029, rotation error = 6.283, time/batch = 1.072
165480/183960, translation error = 11.030, rotation error = 6.286, time/batch = 1.074
165900/183960, translation error = 11.013, rotation error = 6.305, time/batch = 1.074
166320/183960, translation error = 11.000, rotation error = 6.309, time/batch = 1.077
166740/183960, translation error = 10.990, rotation error = 6.301, time/batch = 1.078
167160/183960, translation error = 10.986, rotation error = 6.296, time/batch = 1.074
167580/183960, translation error = 10.984, rotation error = 6.298, time/batch = 1.074
168000/183960, translation error = 10.986, rotation error = 6.295, time/batch = 1.075
168420/183960, translation error = 10.989, rotation error = 6.330, time/batch = 1.071
168840/183960, translation error = 10.970, rotation er

In [13]:
sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

median translation error = 2.084
median rotation error = 3.121
mean translation error = 10.984
mean rotation error = 6.551


In [14]:
def evaluate(trans_errors,rot_errors):
    t = dataset.last_indexes
    trans_errors_month = list()
    trans_errors_month.append(trans_errors[:t[0]])
    trans_errors_month.append(trans_errors[t[0]:t[1]])
    trans_errors_month.append(trans_errors[t[1]:t[2]])
    trans_errors_month.append(trans_errors[t[2]:t[3]])
    trans_errors_month.append(trans_errors[t[3]:t[4]])
    trans_errors_month.append(trans_errors[t[4]:t[5]])
    trans_errors_month.append(trans_errors[t[5]:t[6]])
    trans_errors_month.append(trans_errors[t[6]:])

    rot_errors_month = list()
    rot_errors_month.append(rot_errors[:t[0]])
    rot_errors_month.append(rot_errors[t[0]:t[1]])
    rot_errors_month.append(rot_errors[t[1]:t[2]])
    rot_errors_month.append(rot_errors[t[2]:t[3]])
    rot_errors_month.append(rot_errors[t[3]:t[4]])
    rot_errors_month.append(rot_errors[t[4]:t[5]])
    rot_errors_month.append(rot_errors[t[5]:t[6]])
    rot_errors_month.append(rot_errors[t[6]:])
    
    print('================== median translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("median translation error = {:.3f}".format(np.median(trans_errors_i)))
        
    print('================== median rotation error ==================')
    for rot_errors_i in rot_errors_month:
        print("median rotation error = {:.3f}".format(np.median(rot_errors_i)))
    
    print('================== mean translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("mean translation error = {:.3f}".format(np.mean(trans_errors_i)))
        
    print('================== mean rotation error ==================')  
    for rot_errors_i in rot_errors_month:
        print("mean rotation error = {:.3f}".format(np.mean(rot_errors_i)))
        
evaluate(trans_errors,rot_errors)

================== median translation error ==================
median translation error = 1.588
median translation error = 1.702
median translation error = 1.969
median translation error = 1.956
median translation error = 2.056
median translation error = 2.144
median translation error = 3.554
median translation error = 2.780
================== median rotation error ==================
median rotation error = 2.584
median rotation error = 2.794
median rotation error = 2.950
median rotation error = 2.925
median rotation error = 3.105
median rotation error = 3.262
median rotation error = 4.395
median rotation error = 4.155
================== mean translation error ==================
mean translation error = 4.221
mean translation error = 4.448
mean translation error = 13.170
mean translation error = 12.878
mean translation error = 9.518
mean translation error = 11.637
mean translation error = 21.389
mean translation error = 13.546
================== mean rotation error ==================
m